In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn import metrics
import json
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet as wn
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
from nltk.corpus import stopwords
import collections
from itertools import groupby
import pickle
import re
import matplotlib.colors as mcolors
import warnings
import requests
import csv
from sklearn.preprocessing import MultiLabelBinarizer
from functools import reduce
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

### Global Variables & Functions

#### Variables

In [3]:
nouns = {x.name().split('.', 1)[0] for x in wn.all_synsets('n')}

colors = [color[4:] for color in mcolors.TABLEAU_COLORS.keys()]
colors.extend([color for color in mcolors.CSS4_COLORS.keys()])
colors.extend(
    [
        'whitish', 'bluish', 'reddish', 'greenish', 'backish', 'greyish',
        'backish', 'purplish', 'yellowish', 'orangish', 'brownish', 'pinkish'
    ]
)



traits_dict = {
    'Life Form':
    [
        'Tree', 'Shrub', 'Bush', 'Ficus', 'Strangler', 'Liana', 'Parasitic', 'Palm', 'Herbaceous'
    ],
    'Trunk':
    [
        'Trunk', 'Straight', 'Flared', 'Foothills', 'Silt', 'Aerial'
    ],
    'Root':
    [
        'Root', 'Straight', 'Flared', 'Foothills', 'Silt', 'Aerial'
    ],
    'Latex':
    [
        'Latex'
    ],
    'Phyllotaxis': # Leaf Position
    [
        'Phyllotaxis', 'Alternate', 'Whorled', 'Whorls', 'Opposite'
    ],
    'Leaf Composition':
    [
        'Palmate', 'Pinnate', 'Entire', 'Bi-pinnate'
    ],
    'Crown':
    [
        'Crown'
    ],
    'Stem':
    [
        'Stem', 'Circular', 'Square'
    ],
    'Bark':
    [
        'Bark'
    ],
    'Bark Colour':
    [
        'Bark'
    ],
    'Leaf Shape':
    [
        'Simple', 'Bifoliate', 'Trifoliate', 'Digitized', 'Paripinnate', 'Unipinnate', 'Imperipinnate', 
        'Alternate', 'Bipinnate', 'Pinnate', 'Elliptic', 'Elongate', 'Ovate', 'Round', 'Obovate', 'Lanceolate',
        'Kidney-shaped', 'Heart-shaped', 'Spathulate'
    ],
    'Petiole':
    [
        'Petiole', 'Sessile', 'Petiolated', 'Canaliculate', 'Glands', 'Glandular', 
     'Winged' 'Wings', 'Hairs', 'Hair', 'Translucent'
     ],
    'Leaf Colour':
    [
        'Leaf Colour', 'Leaf Color'
    ],
    'Leaf Blade':
    [
        'Leaf Blade', 'Linear', 'Lanceolate', 'Elliptical', 'Obovate', 'Obtriangular', 
        'Obtriangular', 'Asymmetrical', 'Orbicular', 'Bilobed', 'Lobed', 'Lobes', 'Lobe'
    ],
    'Leaf Base':
    [
        'Leaf Base', 'Rounded', 'Cordate', 'Glands'
    ],
    'Leaf Margin':
    [
        'Margin', 'Smooth', 'Wavy', 'Crenate', 'Toothed', 'Teeth', 'Crenate', 'Serrate'
    ],
    'Leaf Apex':
    [
        'Apex', 'Acuminate', 'Apiculate', 'Mucronate', 'Rounded', 'Emarginated'
    ],
    'Leaf side':
    [
        'Glabrous', 'Pubescent', 'Salt Crystals', 'Scales', 'Woolly', 'Powdery'
    ],
    'Leaf glands':
    [
        'Glands', 'Gland', 'Translucent'
    ],
    'Rachis':
    [
        'Rachis', 'Winged'
    ],
    'Vein':
    [
        'Vein'
    ],
    'Tendril':
    [
        'Tendril'
    ],
    'Spine':
    [
        'Spine', 'Prickle', 'Spines', 'Prickles'
    ],
    'Thornes':
    [
        'Thorn', 'Thornes'
    ],
    'Blade Colour':
    [
        'Blade'
    ],
    'Fruit':
    [
        'Drupe', 'Berry', 'Capsule', 'Pod', 'Follicle', 'Achene', 'Winged', 'Follicle',
        'Pod', 'Nutlet', 'Fruit'
    ],
    'Fruit Shape':
    [
        'locular', 'Globose', 'Flattened', 'Elongate', 'Obovoid', 'Ovate', 'Twisted',
        'Curved', 'Pyriform', 'Ovoid'
    ],
    'Fruit Colour':
    [
        'Fruit'
    ],
    'Inflorescences':
    [
        'Inflorescences', 'Inflorescence', 'Sessile', 'Panicle', 'Flower head', 'Cyme', 'Glomerule', 
        'Fascicle', 'Umbel', 'Corymb', 'Rootlet', 'Spike', 'Dichasium', 'Fascicle',
        'Globose', 'Raceme', 'Fascicle', 'Umbel'
     ],
    'Sexuality':
    [
        'Sexuality', 'Axillary', 'Terminal'
    ],
    'Flower Colour':
    [
        'Flower colour', 'Flower color', 'Flower', 'Flowers'
    ],
    'Flower Shape':
    [
        'Flower shape', 'Petalled', 'Petal', 'Petals', 'Tubular', 'Apetal', 'Butterfly-shaped', 'Shaped', 'Flower', 'Flowers'
    ],
    'Sepal Shape':
    [
        'Sepal', 'Sepals', 'Connate'
    ],
    'Petal Shape':
    [
        'Petal', 'Petals', 'Tepals', 'Tepal', 'Tubular'
    ],
    'Aril Colour':
    [
        'Aril'
    ],
    'Seed Colour':
    [
        'Seed', 
    ],
    'Conspicuousness':
    [
        'Conspicuousness', 'Cryptic'
    ]
}

with open('../../data/supportive/traits_Pierre_and_Andrei.json', 'w') as f:
    json.dump(traits_dict, f)
with open('../../data/supportive/colour_list.json', 'w') as f:
    json.dump(colors, f)

traits_list = list(traits_dict.keys())
traits_list += [trait.lower() for lst in list(traits_dict.values()) for trait in lst]

#### Functions

In [4]:
def corresponding_keys(val, dictionary):
    """returns the corresponding key of a single value 
    assuming the values are lists.

    Args:
        val (string): string present in the dict
        dictionary (dict): dict with lists of stings as values

    Returns:
        list: list of matching keys
    """
    # Init list
    keys = []
    # Search the dict
    for k, v in dictionary.items():
        if val in v:
            keys.append(k)
    return keys
    
def jaccard_similarity(A, B):
    """Calculates the Jaccard similarity two sets.

    Args:
        A (Set): Set A
        B (Set): Set B

    Returns:
        Integer: 0.00 - 1.00
    """

    # Create sets just in case
    A = set(A)
    B = set(B)
    
    # Get intersection of two sets
    nominator = A.intersection(B)

    # Find union of two sets
    denominator = A.union(B)

    # Take the ratio of sizes
    similarity = len(nominator)/len(denominator)

    return similarity


def similarity(groundtruth, pred):
    """Calculates the normal similarity between two sets.

    Args:
        A (Set): Set A
        B (Set): Set B

    Returns:
        Integer: 0.00 - 1.00
    """

    # Create sets just in case
    groundtruth = set(groundtruth)
    pred = set(pred)

    # Find intersection of two sets
    nominator = groundtruth.intersection(pred)

    # Find union of two sets
    denominator = groundtruth

    # Take the ratio of sizes
    similarity = len(nominator)/(len(denominator))

    return similarity

def resentesize(lst):

    sentence = ''

    # Loop every 2 items 
    for item1, item2 in zip(lst[::2], lst[1::2]):
        
        # Set verb
        verb = 'has'
        chunk = item2.split(' ')[-1]
        if chunk not in nouns or chunk in colors:
            verb = 'is'

        sentence += f'{item1.capitalize()} {verb} {item2}. '
    
    return sentence


def get_wiki_main_image(title):
    url = 'https://en.wikipedia.org/w/api.php'
    data = {
        'action' :'query',
        'format' : 'json',
        'formatversion' : 2,
        'prop' : 'pageimages|pageterms',
        'piprop' : 'original',
        'titles' : title
    }
    response = requests.get(url, data)
    json_data = json.loads(response.text)
    return json_data['query']['pages'][0]['original']['source'] if len(json_data['query']['pages']) >0 else 'Not found'


### Load GT Data

In [5]:
root = "../../data/external/"

#### Data Andrei

In [6]:
file_name = root + 'Dataset_Andrei.csv'
df_Andrei = pd.read_csv(file_name)

# Get Dummies to match DF Pierre
df_Andrei_dummies = pd.get_dummies(df_Andrei.iloc[:, 2:])
# Set species back
df_Andrei_dummies = df_Andrei_dummies.set_index(df_Andrei['Species'])

# Create tuple list for multi index
Andrei_multi_index = []
for top_index in df_Andrei.columns:
    for sub_index in df_Andrei_dummies.columns:
        if top_index in sub_index:

            sub_index = sub_index.split('_')[-1]
            Andrei_multi_index.append((top_index, sub_index))

# Set Mutli index
df_Andrei_dummies.columns = pd.MultiIndex.from_tuples(Andrei_multi_index)
df_Andrei_dummies

Life form      Leaf position                      \
                               liana tree     alternate alternate, opposite   
Species                                                                       
Avicennia germinans                0    1             0                   0   
Metopium brownei                   0    1             1                   0   
Handroanthus billbergii            0    1             0                   0   
Bourreria succulenta               0    1             1                   0   
Bursera karsteniana                0    1             1                   0   
Bursera simaruba                   0    1             1                   0   
Bursera tomentosa                  0    1             1                   0   
Cynophalla flexuosa                1    0             1                   0   
Cynophalla hastata                 0    1             1                   0   
Quadrella indica                   0    1             1                   0   
Quadrella odoratissima             0    1             1                   0   
Crossopetalum rhacoma              0    1             0                   0   
Maytenus tetragona                 0    1             1                   0   
Maytenus versluysii                0    1             1                   0   
Clusia rosea                       0    1             0                   0   
Conocarpus erectus                 0    1             1                   0   
Laguncularia racemosa              0    1             0                   0   
Erythroxylum havanense             0    1             1                   0   
Croton niveus                      0    1             1                   0   
Hippomane mancinella               0    1             1                   0   
Libidibia coriaria                 0    1             1                   0   
Pithecellobium unguis-cati         0    1             1                   0   
Prosopis juliflora                 0    1             1                   0   
Vachellia tortuosa                 0    1             1                   0   
Vitex compressa                    0    1             0                   0   
Vitex cymosa                       0    1             0                   0   
Trichilia trifolia                 0    1             1                   0   
Eugenia procera                    0    1             0                   0   
Guapira pacurero                   0    1             0                   0   
Coccoloba swartzii                 0    1             1                   0   
Coccoloba uvifera                  0    1             1                   0   
Krugiodendron ferreum              0    1             0                   1   
Rhizophora mangle                  0    1             0                   0   
Guettarda roupalifolia             0    1             0                   0   
Randia aculeata                    0    1             0                   0   
Amyris ignea                       0    1             1                   0   
Zanthoxylum monophyllum            0    1             1                   0   
Casearia tremula                   0    1             1                   0   
Schoepfia schreberi                0    1             1                   0   
Jacquinia arborea                  0    1             1                   0   
Guaiacum officinale                0    1             0                   0   
Guaiacum sanctum                   0    1             0                   0   

                                                           \
                           opposite opposite, whorls of 3   
Species                                                     
Avicennia germinans               1                     0   
Metopium brownei                  0                     0   
Handroanthus billbergii           1                     0   
Bourreria succulenta              0                     0   
Bursera karsteniana               0         

#### Data Pierre

In [7]:
file_name = root + 'Dataset_Pierre.csv'
df_Pierre = pd.read_csv(file_name, header=[0, 1]) 
df_Pierre = df_Pierre.iloc[: , 1:]
df_Pierre = df_Pierre.set_index(df_Pierre['Species']['species'])
df_Pierre = df_Pierre.drop(columns=['Species'])
df_Pierre

/var/folders/0p/7b4jnnp15d5gx16h36l4ycx00000gn/T/ipykernel_31715/3001850027.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_Pierre = df_Pierre.drop(columns=['Species'])


plant type                                   \
                                  Tree Shrub Bush Ficus strangler Liana   
species                                                                   
Acacia amythethophylla               1     1    0               0     0   
Acacia ataxacantha                   1     1    1               0     1   
Acacia dudgeoni                      1     1    0               0     0   
Acacia ehrenbergiana                 1     1    0               0     0   
Acacia erythrocalyx                  0     0    1               0     1   
...                                ...   ...  ...             ...   ...   
Ziziphus abyssinica                  1     1    1               0     0   
Ziziphus lotus ssp. saharae          0     1    1               0     0   
Ziziphus mauritiana                  1     1    1               0     0   
Ziziphus mucronata                   1     1    1               0     0   
Ziziphus spina-christi               0     1    1               0     0   

                                                            \
                            Parasitic Palm tree Herbaceous   
species                                                      
Acacia amythethophylla              0         0          0   
Acacia ataxacantha                  0         0          0   
Acacia dudgeoni                     0         0          0   
Acacia ehrenbergiana                0         0          0   
Acacia erythrocalyx                 0         0          0   
...                               ...       ...        ...   
Ziziphus abyssinica                 0         0          0   
Ziziphus lotus ssp. saharae         0         0          0   
Ziziphus mauritiana                 0         0          0   
Ziziphus mucronata                  0         0          0   
Ziziphus spina-christi              0         0          0   

                                      phyllotaxis                       ...  \
                            Phyllotaxis alternate Opposite phyllotaxis  ...   
species                                                                 ...   
Acacia amythethophylla                          1                    0  ...   
Acacia ataxacantha                              1                    0  ...   
Acacia dudgeoni                                 1                    0  ...   
Acacia ehrenbergiana                            1                    0  ...   
Acacia erythrocalyx                             1                    0  ...   
...                                           ...                  ...  ...   
Ziziphus abyssinica                             1                    0  ...   
Ziziphus lotus ssp. saharae                     1                    0  ...   
Ziziphus mauritiana                             1                    0  ...   
Ziziphus mucronata                              1                    0  ...   
Ziziphus spina-christi                          1                    0  ...   

                                      flower color                        \
                            Yellow coloured flower Green coloured flower   
species                                                                    
Acacia amythethophylla                           1                     0   
Acacia ataxacantha                               1                     0   
Acacia dudgeoni                                  1                     0   
Acacia ehrenbergiana                             1                     0   
Acacia erythrocalyx                              0                     0   
...                                            ...                   ...   
Ziziphus abyssinica                              1                     0   
Ziziphus lotus ssp. saharae                      1                     0   
Ziziphus mauritiana                              1                     0   
Ziziphus mucronata                               1                     0   
Ziziphus spin

#### Data Palms

In [8]:
file_name = root + 'Dataset_Kissling.txt'
df_Daniel = pd.read_csv(file_name,
                 sep='\t', encoding='Latin-1')
palm_species = df_Daniel[~df_Daniel.isnull().any(axis=1)]['SpecName'].values
df_Daniel.set_index('SpecName', inplace=True)

### RESHAPE DATA FOR PALMS

# Exclude string types
df_Daniels_int = df_Daniel.select_dtypes(exclude=[object])
df_Daniels_str = df_Daniel.select_dtypes(include=[object])
# Drop numbers
df_Daniels_semi_ints = df_Daniels_int.loc[:, df_Daniels_int.max() <= 3]
# Merge again
df_Daniel_edit = pd.merge(df_Daniels_str, df_Daniels_semi_ints, left_index=True, right_index=True)

# Get colors as lst of lsts 
FruitColorDescription_colors_lst = []

for palm_colors in df_Daniels_str['FruitColorDescription'].values:
    if type(palm_colors) == str:
        #print(type(colors))
        palm_colors = re.split(r'; |to | |-', palm_colors)

        #print(palm_colors)
        FruitColorDescription_colors_lst.append([color for color in palm_colors if color in colors])
    else:
        FruitColorDescription_colors_lst.append([])

MainFruitColors_colors_lst = []

for palm_colors in df_Daniels_str['MainFruitColors'].values:
    if type(palm_colors) == str:
        #print(type(colors))
        palm_colors = re.split(r'; |to | |-', palm_colors)

        #print(palm_colors)
        MainFruitColors_colors_lst.append([color for color in palm_colors if color in colors])
    else:
        MainFruitColors_colors_lst.append([])

# Init SKlearn MLB
mlb = MultiLabelBinarizer()

# Create dummies for color columns
df_FruitColorDescription = pd.DataFrame(
    {
        'FruitColorDescription': FruitColorDescription_colors_lst
    }, columns=['FruitColorDescription'])

s = df_FruitColorDescription['FruitColorDescription']
df_FruitColorDescription = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df_Daniel.index)

# Multiindex columns
columns = [('Fruit Colour Description', column) for column in df_FruitColorDescription.columns]
df_FruitColorDescription.columns = pd.MultiIndex.from_tuples(columns)

# Create dummies for color columns
df_MainFruitColors = pd.DataFrame(
    {
        'MainFruitColors': MainFruitColors_colors_lst
    }, columns=['MainFruitColors'])

s = df_MainFruitColors['MainFruitColors']
df_MainFruitColors = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df_Daniel.index)

# Multiindex columns
columns = [('Fruit Colour', column) for column in df_MainFruitColors.columns]
df_MainFruitColors.columns = pd.MultiIndex.from_tuples(columns)

df_Daniels_str_non_color = df_Daniels_str[['UnderstoreyCanopy', 'FruitSizeCategorical', 'FruitShape', 'Conspicuousness']]

# df_Daniels_str_non_color.columns = pd.MultiIndex.from_tuples(
#     [
#         ('Crown', 'UnderstoreyCanopy'),
#         ('Fruit Size', 'FruitSizeCategorical'),
#         ('Fruit Shape', 'FruitShape'),
#         ('Conspicuousness', 'Conspicuousness'),
#     ]
# )

df_Daniels_str_non_color_dummies = pd.get_dummies(df_Daniels_str_non_color)
columns = []
for column in df_Daniels_str_non_color_dummies.columns:
    level0, level1 = column.split('_')
    if level0 == 'UnderstoreyCanopy':
        level0 = 'Crown'
    elif level0 == 'FruitSizeCategorical':
        level0 = 'Fruit Size'
    elif level0 == 'FruitShape':
        level0 = 'Fruit Shape'
    elif level0 == 'Conspicuousness':
        level0 = 'Conspicuousness'
    columns.append((level0, level1))
    
df_Daniels_str_non_color_dummies.columns = pd.MultiIndex.from_tuples(columns)

### JOIN ALL DATA
data_frames = [df_FruitColorDescription, df_MainFruitColors, df_Daniels_str_non_color_dummies]
df_Daniel_merged = pd.concat(data_frames, axis=1)
df_Daniel_merged

Fruit Colour Description                             \
                                            black blue bluish brown brownish   
SpecName                                                                       
Acanthophoenix crinita                          1    0      0     0        0   
Acanthophoenix rousselii                        1    0      0     0        0   
Acanthophoenix rubra                            1    0      0     0        0   
Acoelorrhaphe wrightii                          1    0      0     1        0   
Acrocomia aculeata                              0    0      0     0        0   
...                                           ...  ...    ...   ...      ...   
Wettinia quinaria                               0    0      0     1        0   
Wettinia radiata                                0    0      0     0        0   
Wettinia verruculosa                            0    0      0     1        0   
Wodyetia bifurcata                              0    0      0     0        0   
Zombia antillarum                               0    0      0     0        0   

                                                                  ...  \
                         chocolate coral crimson darkgreen green  ...   
SpecName                                                          ...   
Acanthophoenix crinita           0     0       0         0     0  ...   
Acanthophoenix rousselii         0     0       0         0     0  ...   
Acanthophoenix rubra             0     0       0         0     0  ...   
Acoelorrhaphe wrightii           0     0       0         0     0  ...   
Acrocomia aculeata               0     0       0         0     1  ...   
...                            ...   ...     ...       ...   ...  ...   
Wettinia quinaria                0     0       0         0     1  ...   
Wettinia radiata                 0     0       0         0     0  ...   
Wettinia verruculosa             0     0       0         0     1  ...   
Wodyetia bifurcata               0     0       0         0     0  ...   
Zombia antillarum                0     0       0         0     0  ...   

                         Fruit Size Fruit Shape                            \
                              small   ellipsoid elongate fusiform globose   
SpecName                                                                    
Acanthophoenix crinita            1           0        0        0       0   
Acanthophoenix rousselii          1           0        0        0       0   
Acanthophoenix rubra              1           0        0        0       0   
Acoelorrhaphe wrightii            1           0        0        0       0   
Acrocomia aculeata                0           0        0        0       0   
...                             ...         ...      ...      ...     ...   
Wettinia quinaria                 1           0        0        0       0   
Wettinia radiata                  0           0        0        0       1   
Wettinia verruculosa              1           0        0        0       0   
Wodyetia bifurcata                0           0        0        0       1   
Zombia antillarum                 1           0        0        0       0   

                                                 Conspicuousness          
                         ovoid pyramidal rounded     conspicuous cryptic  
SpecName                                                                  
Acanthophoenix crinita       0         0       0               0       1  
Acanthophoenix rousselii     1         0       0               0       1  
Acanthophoenix rubra         1         0       0               0       1  
Acoelorrhaphe wrightii       1         0       0               0       1  
Acrocomia aculeata           1         0       0               0       1  
...                        ...       ...     ...             ...     ...  
Wettinia quinaria            1         0       0               0       1  
Wettinia radiata             0         0       0    

In [19]:
df_Daniels_int.loc[:, df_Daniels_int.max() >= 3]

,MaxStemHeight_m,MaxStemDia_cm,MaxLeafNumber,Max_Blade_Length_m,Max_Rachis_Length_m,Max_Petiole_length_m,AverageFruitLength_cm,MinFruitLength_cm,MaxFruitLength_cm,AverageFruitWidth_cm,MinFruitWidth_cm,MaxFruitWidth_cm
SpecName,,,,,,,,,,,,
Acanthophoenix crinita,10.0,20.0,15.0,2.30,NaN,NaN,0.65,0.6,0.7,0.50,NaN,NaN
Acanthophoenix rousselii,25.0,30.0,NaN,3.00,NaN,NaN,2.00,NaN,NaN,0.80,NaN,NaN
Acanthophoenix rubra,15.0,18.0,20.0,3.10,3.0,NaN,1.00,NaN,NaN,0.70,NaN,NaN
Acoelorrhaphe wrightii,9.1,15.0,25.0,1.30,0.7,0.65,0.70,NaN,NaN,0.70,0.5,0.9
Acrocomia aculeata,12.0,50.0,30.0,3.50,2.5,NaN,4.25,3.5,5.0,4.60,3.8,5.4
...,...,...,...,...,...,...,...,...,...,...,...,...
Wettinia quinaria,15.0,20.0,6.0,5.50,3.2,1.35,2.75,2.0,3.5,2.00,1.5,2.5
Wettinia radiata,9.0,10.0,6.0,4.33,3.4,NaN,4.00,3.0,4.0,2.00,NaN,NaN
Wettinia verruculosa,10.0,20.0,6.0,6.00,3.7,2.40,2.50,NaN,NaN,1.50,NaN,NaN


## Prediction Data

### Load Data

In [9]:
root = "../../data/processed/"
sentences_all = {}

f = open(F"{root}Sentences_Pierre.pkl", 'rb')
sentences_Pierre = pickle.load(f)
sentences_all |= sentences_Pierre 

f = open(F"{root}Sentences_Andrei.pkl", 'rb')
sentences_Andrei = pickle.load(f)
sentences_all |= sentences_Andrei 

f = open(F"{root}Sentences_Kissling.pkl", 'rb')
sentences_Kissling = pickle.load(f)
sentences_all |= sentences_Kissling 

# Drop duplicates
for species, sentences in sentences_all.items():
    sentences_all[species] = list(set(sentences))


### Match Data

In [16]:
k= 5
google_form_lst = []

for idx, (species, sentences) in enumerate(tqdm(sentences_all.items())):

    # if idx < 400:
    #     continue
    
    df_select = [df_Andrei_dummies, df_Pierre, df_Daniel_merged]
    if species in df_Andrei_dummies.index:
        df_select = df_select[0]
    elif species in df_Pierre.index:
        df_select = df_select[1]
    elif species in df_Daniel_merged.index:
        df_select = df_select[2]
    else:
        raise ValueError

    for gt_main_trait in df_select.columns.get_level_values(0).unique():
        df_subset = df_select[df_select.index == species][gt_main_trait]
        present_traits = df_subset.loc[:, df_subset.any()].columns.values
        df_sent  = ' '.join(gt_main_trait + ' ' + present_traits)

        top_list = []

        for sentence in sentences:
            gt_sim = jaccard_similarity(sentence, df_sent)
            top_list.append((gt_sim, sentence))

        top_list.sort(reverse=True)
        top_k_list = [sentence for (_, sentence) in top_list[0:k]]

        google_form_lst.append((species, gt_main_trait, gt_main_trait, list(present_traits), *top_k_list))

100%|██████████| 647/647 [00:15<00:00, 42.16it/s]


### Create Subset

In [17]:
# Google forms
df_google = pd.DataFrame(google_form_lst, columns=['Species', 'Main Trait', 'GT Main Trait', 'GT Sub Traits', '1', '2', '3', '4', '5'])
df_google

,Species,Main Trait,GT Main Trait,GT Sub Traits,1,2,3,4,5
0,Acacia amythethophylla,plant type,plant type,"[Tree, Shrub]",Shrub or tree to 7 m. Thorns in pairs at nodes...,The branch bark is apparently 1.5% approx.,"Perennial, Not climbing, Shrub/Tree.",Flower-heads abundant golden yellow.,"Inflorescence large, terminal, much-branched, ..."
1,Acacia amythethophylla,phyllotaxis,phyllotaxis,[Phyllotaxis alternate],The branch bark is apparently 1.5% approx.,"Pinnae sometimes 20 or more pairs, with 50 or ...","Inflorescence large, terminal, much-branched, ...",Acacia confusa - Stem Smaller peaks suspected ...,Flower-heads abundant golden yellow.
2,Acacia amythethophylla,trunk and root,trunk and root,[Base of trunk straight],"Fruits flat, reddish-brown, remaining on the t...","Fruit flat, dehiscent.",Young plants 6-8 months old from seed.,Shrub or tree to 7 m. Thorns in pairs at nodes...,Flower-heads abundant golden yellow.
3,Acacia amythethophylla,latex,latex,[No exudate],Flower-heads abundant golden yellow.,"Fruit flat, dehiscent.",Young plants 6-8 months old from seed.,"Perennial, Not climbing, Shrub/Tree.","Fruits flat, reddish-brown, remaining on the t..."
4,Acacia amythethophylla,crown,crown,[Horizontally spreading crowns],"Inflorescence large, terminal, much-branched, ...","Stipules spinescent, stout, brown, glossy, com...","Flowers orange or yellow, strongly and sweetly...",Flower-heads abundant golden yellow.,Acacia confusa - Stem Smaller peaks suspected ...
...,...,...,...,...,...,...,...,...,...
11077,Wodyetia bifurcata,Fruit Colour,Fruit Colour,[red],Fruits close up.,Fruit on ground below Foxtail.,Foxtail Palm Tree: Medium.,Fruits : 2 inches long.,"Fruits are 2 inches long, egg shaped, and oran..."
11078,Wodyetia bifurcata,Crown,Crown,[canopy],Capi Nature Row Egg Antraciet.,Leaf scars are regular and closely spaced.,Leaves are very plumose with leaflets coming o...,The crownshaft of the foxtail palm is light to...,Nous contacter.
11079,Wodyetia bifurcata,Fruit Size,Fruit Size,[large],Fruit on ground below Foxtail.,Foxtail Palm Tree: Large Multi Stem.,Immature green fruit of the Foxtail Palm.,Fruit showing remains of style.,Foxtail Palm Tree: Small.
11080,Wodyetia bifurcata,Fruit Shape,Fruit Shape,[globose],Washingtonia robusta Agave americana Scroll to...,Leaflets are thin and about a foot long.,Fruits close up.,"Fruits are 2 inches long, egg shaped, and oran...",The fruits grow plentiful on the stalk.


In [12]:
df_google_subset = df_google\
                    .dropna()\
                    .sample(n=20, axis=0, random_state=333)\

df_google_subset

,Species,Main Trait,GT Main Trait,GT Sub Traits,1,2,3,4,5
273,Chamaedorea allenii,Fruit Size,Fruit Size,[small],"Subshrub, Solitary palm.","Small, sometimes moderate, erect or procumbent...","Seed erect, globose, or ellipsoidal, hilum sma...","Fruit small, globose or oblong, stigmatic rema...",Flowers sessile or partly enclosed in a cavity...
101,Astrocaryum vulgare,Conspicuousness,Conspicuousness,[conspicuous],The trunk is heavily spined and even the inflo...,The seed is covered externally with an orange ...,The trunk and stem parts have large black spin...,"The spadices are held erect, and the inner spa...","The trunk and stem parts, have large black spi..."
647,Dypsis tenuissima,Conspicuousness,Conspicuousness,[cryptic],"Infloresence interfoliar, unbranched, pistilla...",Clustering palm in groups of 2-6.,Definitie: An autotroph which is capable of tr...,Definition: whole plant arborescent.,Groundcovers.
430,Cyrtostachys renda,Fruit Shape,Fruit Shape,[ovoid],"Some specimens were spectacular, while some we...","Solitary or clustered, moderate to tall, unarm...",The seed is brownish in color with an oval non...,"Si riproduce per divisione e per seme, che man...","Stems erect, bare, conspicuously ringed with l..."
1437,Voanioala gerardii,Fruit Size,Fruit Size,[large],Voanioala can reach up to 5 meters in height.,"Solitary, unarmed, pleonanthic monoecious palms.",It is a rediculously slow growing palm.,V. gerondii is polypoid and has a chromosome c...,The flower bearing branches are numerous and b...
517,Dypsis catatiana,Fruit Colour,Fruit Colour,[red],Entire bifid or pinnately divided leaves.,Solitary small or dwarf palm.,"Solitary dwarf palm, to 1 m tall.","A small, solitary palm, to about 1.5m tall, wi...",Clear.
224,Basselinia pancheri,Crown,Crown,[canopy],Crownshaft and flower spathe.,"Crownshaft detail, with inflorescence.","Almost entire, stffly erect leaves, with very ...","Small to stout, solitary or clustered, unarmed...",Characteristic small to moderate pinnate-leave...
946,Mauritiella armata,Fruit Shape,Fruit Shape,[globose],Mauritiella is a dioecious genus of flowering ...,"This is round to ellipsoidal, located basally.",Female plant with two inflorescences and one f...,Definition: whole plant herbaceous.,"The leaves are fairly stiff and deeply split, ..."
708,Geonoma deversa,Fruit Colour Description,Fruit Colour Description,"[black, purplish]","Shrub, Caespitose palm.",Definitie: An autotroph which is capable of tr...,"Small to moderate, solitary or clustered, unar...","Fruit Â±globose, sometimes somewhat pointed, g...","Seed Â± globose, hilum short, basal, raphe enc..."
346,Chamaedorea pauciflora,Fruit Shape,Fruit Shape,[ovoid],Flowers sessile or partly enclosed in a cavity...,"Stem usually slender, covered wholly or partia...","Herb, Subshrub, Solitary palm.","A small, understory palm, with entire, deeply ...","Some features of floral anatomy, including vas..."


### Add Images

In [13]:
random_sample_species = df_google_subset['Species'].values

images_links = {}
for species in tqdm(random_sample_species):
    species_ = species.replace(' ', '_')
    try:
        img_url = get_wiki_main_image(species_)
        if img_url[-3:] != 'jpg':
            img_url = 'https://upload.wikimedia.org/wikipedia/commons/1/14/No_Image_Available.jpg'
        # if img_url[-3:].isupper():
        #     img_url = 'https://upload.wikimedia.org/wikipedia/commons/1/14/No_Image_Available.jpg'
        images_links[species] = img_url
    except:
        images_links[species] = 'https://upload.wikimedia.org/wikipedia/commons/1/14/No_Image_Available.jpg'

# df_google_subset = df_google[df_google['Species'].isin(random_sample_species)]
df_google_subset = df_google_subset.set_index('Species')
df_google_subset["URL"] = pd.Series(images_links)
df_google_subset.to_csv(f'{root}top_sents_all_AllSentencesAgainstTrait_Random20Subset.csv', sep='\t')#, quoting=csv.QUOTE_ALL)

100%|██████████| 20/20 [00:03<00:00,  5.69it/s]


In [14]:
df_google_subset

,Main Trait,GT Main Trait,GT Sub Traits,1,2,3,4,5,URL
Species,,,,,,,,,
Chamaedorea allenii,Fruit Size,Fruit Size,[small],"Subshrub, Solitary palm.","Small, sometimes moderate, erect or procumbent...","Seed erect, globose, or ellipsoidal, hilum sma...","Fruit small, globose or oblong, stigmatic rema...",Flowers sessile or partly enclosed in a cavity...,https://upload.wikimedia.org/wikipedia/commons...
Astrocaryum vulgare,Conspicuousness,Conspicuousness,[conspicuous],The trunk is heavily spined and even the inflo...,The seed is covered externally with an orange ...,The trunk and stem parts have large black spin...,"The spadices are held erect, and the inner spa...","The trunk and stem parts, have large black spi...",https://upload.wikimedia.org/wikipedia/commons...
Dypsis tenuissima,Conspicuousness,Conspicuousness,[cryptic],"Infloresence interfoliar, unbranched, pistilla...",Clustering palm in groups of 2-6.,Definitie: An autotroph which is capable of tr...,Definition: whole plant arborescent.,Groundcovers.,https://upload.wikimedia.org/wikipedia/commons...
Cyrtostachys renda,Fruit Shape,Fruit Shape,[ovoid],"Some specimens were spectacular, while some we...","Solitary or clustered, moderate to tall, unarm...",The seed is brownish in color with an oval non...,"Si riproduce per divisione e per seme, che man...","Stems erect, bare, conspicuously ringed with l...",https://upload.wikimedia.org/wikipedia/commons...
Voanioala gerardii,Fruit Size,Fruit Size,[large],Voanioala can reach up to 5 meters in height.,"Solitary, unarmed, pleonanthic monoecious palms.",It is a rediculously slow growing palm.,V. gerondii is polypoid and has a chromosome c...,The flower bearing branches are numerous and b...,https://upload.wikimedia.org/wikipedia/commons...
Dypsis catatiana,Fruit Colour,Fruit Colour,[red],Entire bifid or pinnately divided leaves.,Solitary small or dwarf palm.,"Solitary dwarf palm, to 1 m tall.","A small, solitary palm, to about 1.5m tall, wi...",Clear.,https://upload.wikimedia.org/wikipedia/commons...
Basselinia pancheri,Crown,Crown,[canopy],Crownshaft and flower spathe.,"Crownshaft detail, with inflorescence.","Almost entire, stffly erect leaves, with very ...","Small to stout, solitary or clustered, unarmed...",Characteristic small to moderate pinnate-leave...,https://upload.wikimedia.org/wikipedia/commons...
Mauritiella armata,Fruit Shape,Fruit Shape,[globose],Mauritiella is a dioecious genus of flowering ...,"This is round to ellipsoidal, located basally.",Female plant with two inflorescences and one f...,Definition: whole plant herbaceous.,"The leaves are fairly stiff and deeply split, ...",https://upload.wikimedia.org/wikipedia/commons...
Geonoma deversa,Fruit Colour Description,Fruit Colour Description,"[black, purplish]","Shrub, Caespitose palm.",Definitie: An autotroph which is capable of tr...,"Small to moderate, solitary or clustered, unar...","Fruit Â±globose, sometimes somewhat pointed, g...","Seed Â± globose, hilum short, basal, raphe enc...",https://upload.wikimedia.org/wikipedia/commons...
